In [186]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')

In [199]:
season = 2020
# csv to save
csv_file = '../' + 'Odds' + '/' + str(season) + '_odds_ds'+ '.csv'
print(csv_file)
odds_df = pd.read_csv(csv_file)

../Odds/2020_odds_ds.csv


In [200]:
odds_df.head(5)

,id,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,season,bookmaker,winner_home,winner_away,winner_draw,double_home_away,double_home_draw,double_away_draw,week_y
0,605124.0,533,532,6,-0.230920,1.042431,2.308306,1,1,140,2020,8,1.60,5.25,4.1,1.22,1.17,2.25,6.0
1,605117.0,543,548,6,0.029372,1.048234,2.271473,-1,1,140,2020,8,2.60,2.63,3.4,1.30,1.44,1.50,6.0
2,605119.0,545,727,6,1.182168,1.240975,1.048746,0,0,140,2020,8,2.70,2.90,3.0,1.36,1.40,1.44,6.0
3,605115.0,542,797,6,0.673380,2.285670,0.538878,-1,0,140,2020,8,1.75,5.50,3.3,1.33,1.14,2.00,6.0
4,605116.0,531,539,6,1.218447,1.058653,2.199194,1,1,140,2020,8,2.00,4.00,3.3,1.33,1.22,1.73,6.0


In [201]:
def get_max_favorite(row):
    if row["winner_home"] > row["winner_away"] and row["winner_home"] > row["winner_draw"]:
        return 1
    elif row["winner_away"] > row["winner_home"] and row["winner_away"] > row["winner_draw"]:
        return -1
    elif (row["winner_draw"] > row["winner_away"] and row["winner_draw"] > row["winner_home"]) or row["winner_home"] == row["winner_away"]:
        return 0

    # Expect to normalize probabilities by margin, by now returns the max probability
def get_favorite(df):
    fav = []
    for index, row in df.iterrows():
        fav.append(get_max_favorite(row))
    return fav

def prob(x):
    return (1/x)

# TODO: Resolve the normalized margin situation
def feature_engineer(odds):
    # Odds as probability (inverse prob)
    odds["winner_home"] = odds["winner_home"].apply(prob)
    odds["winner_away"] = odds["winner_away"].apply(prob)
    odds["winner_draw"] = odds["winner_draw"].apply(prob)
    odds["double_home_away"] = odds["double_home_away"].apply(prob)
    odds["double_home_draw"] = odds["double_home_draw"].apply(prob)
    odds["double_away_draw"] = odds["double_away_draw"].apply(prob)
    # Calculate margin for winner 1 - (sum of winner)
    odds["winner_margin"] = abs(1 - (odds["winner_home"] + odds["winner_away"] + odds["winner_draw"]))
    # Calculate winner (the favorite of the bookmaker)
    odds["favorite"] = get_favorite(odds)
    # If it is underdog (predict is different than the favorite)
    odds["underdog"] = (odds["favorite"] != odds["predict"]).astype(int)
    # Calculate margin for double --> TODO: try a normalization 
    # Each has a margin
    odds["double_margin_ha"] = odds["double_home_away"] - odds["winner_home"] - odds["winner_away"]
    odds["double_margin_hd"] = odds["double_home_draw"] - odds["winner_home"] - odds["winner_draw"]
    odds["double_margin_ad"] = odds["double_away_draw"] - odds["winner_away"] - odds["winner_draw"]
    # Calculate doble on predict
    odds["predict_home_away"] = odds["predict_win"] + odds["predict_lose"]
    odds["predict_home_draw"] = odds["predict_win"] + odds["predict_draw"]
    odds["predict_away_draw"] = odds["predict_lose"] + odds["predict_draw"]
    # Dropping bookmaker, season, id, week-y (is not completely full)
    odds = odds.drop(columns=['bookmaker', 'season', 'id', 'week_y'])
    return odds

In [202]:
transformed_odds = feature_engineer(odds_df)

In [203]:
transformed_odds.head(5)

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,winner_home,...,double_away_draw,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw
0,533,532,6,-0.230920,1.042431,2.308306,1,1,140,0.625000,...,0.444444,0.059379,1,0,0.004196,-0.014202,0.010066,2.077386,3.350737,0.811511
1,543,548,6,0.029372,1.048234,2.271473,-1,1,140,0.384615,...,0.666667,0.058961,1,0,0.004387,0.015711,-0.007679,2.300845,3.319706,1.077605
2,545,727,6,1.182168,1.240975,1.048746,0,0,140,0.370370,...,0.694444,0.048531,1,1,0.020096,0.010582,0.016284,2.230914,2.289721,2.423143
3,542,797,6,0.673380,2.285670,0.538878,-1,0,140,0.571429,...,0.500000,0.056277,1,1,-0.001367,0.002734,0.015152,1.212259,2.824548,2.959050
4,531,539,6,1.218447,1.058653,2.199194,1,1,140,0.500000,...,0.578035,0.053030,1,0,0.001880,0.016642,0.025004,3.417641,3.257847,2.277100


In [204]:
# Target the real
season = 2020
# csv to save
csv_file = '../' + 'Odds' + '/' + str(season) + '_' + '10_29_'+ 'odds_test'+ '.csv'
print(csv_file)
odds_test = pd.read_csv(csv_file)
transformed_odds_test = feature_engineer(odds_test)

../Odds/2020_10_29_odds_test.csv


In [205]:
transformed_odds_test.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,predict,league,winner_home,winner_away,...,double_away_draw,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw
0,542,529,8,2.296468,1.074388,-0.051669,-1,140,0.153846,0.694444,...,0.900901,0.070513,-1,0,-0.000833,0.004160,-0.015766,2.244799,1.022718,3.370856
1,531,536,8,0.805671,1.058271,2.276429,1,140,0.347222,0.380228,...,0.714286,0.060784,-1,1,0.007844,0.013889,0.000724,3.082100,3.334700,1.863943
2,543,797,8,-0.238466,1.091237,2.313058,1,140,0.666667,0.153846,...,0.400000,0.070513,1,0,-0.000841,-0.015766,-0.003846,2.074592,3.404295,0.852772
3,538,548,8,0.963633,1.276102,1.044164,0,140,0.333333,0.420168,...,0.735294,0.066001,-1,1,-0.018207,0.007761,0.002626,2.007796,2.320265,2.239734
4,545,724,8,1.213535,1.049107,2.187189,1,140,0.416667,0.307692,...,0.653595,0.057692,1,0,0.010935,0.001880,0.012569,3.400724,3.236296,2.262641


In [206]:
transformed_odds_test.shape

(76, 23)

In [207]:
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn import preprocessing
def prediction(train, test):
    #nb_ovo=OneVsOneClassifier(GaussianNB())
    nb_ovo=OneVsOneClassifier(RandomForestClassifier(random_state=0, criterion="entropy", max_features="log2", min_samples_leaf=2))
    #nb_ovo=OneVsOneClassifier(SVC(kernel='linear', probability=True))
    target = train['real']
    predict = test['predict']
    favorite = test['favorite']
    train = train.drop(columns=['real'])
    #train = train.drop(columns=['real', 'predict', 'favorite', 'underdog'])
    #test = test.drop(columns=['predict', 'favorite', 'underdog'])
    train_set = preprocessing.StandardScaler().fit_transform(train)
    test_set = preprocessing.StandardScaler().fit_transform(test)
    nb_ovo.fit(train_set, target)
    Y = nb_ovo.predict(test_set)
    test['real'] = Y
    test['predict'] = predict
    test['favorite'] = favorite
    return test

In [208]:
new_prediction = prediction(transformed_odds, transformed_odds_test)

In [209]:
new_prediction.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,predict,league,winner_home,winner_away,...,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw,real
0,542,529,8,2.296468,1.074388,-0.051669,-1,140,0.153846,0.694444,...,0.070513,-1,0,-0.000833,0.004160,-0.015766,2.244799,1.022718,3.370856,-1
1,531,536,8,0.805671,1.058271,2.276429,1,140,0.347222,0.380228,...,0.060784,-1,1,0.007844,0.013889,0.000724,3.082100,3.334700,1.863943,-1
2,543,797,8,-0.238466,1.091237,2.313058,1,140,0.666667,0.153846,...,0.070513,1,0,-0.000841,-0.015766,-0.003846,2.074592,3.404295,0.852772,1
3,538,548,8,0.963633,1.276102,1.044164,0,140,0.333333,0.420168,...,0.066001,-1,1,-0.018207,0.007761,0.002626,2.007796,2.320265,2.239734,1
4,545,724,8,1.213535,1.049107,2.187189,1,140,0.416667,0.307692,...,0.057692,1,0,0.010935,0.001880,0.012569,3.400724,3.236296,2.262641,-1


In [79]:
new_prediction.groupby(['real', 'favorite']).count()

home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real favorite                                                                   
-1   -1                  29            29      29            29            29   
      0                   1             1       1             1             1   
      1                   3             3       3             3             3   
 0   -1                   1             1       1             1             1   
      1                  14            14      14            14            14   
 1    1                  28            28      28            28            28   

               predict_win  predict  league  winner_home  winner_away  \
real favorite                                                           
-1   -1                 29       29      29           29           29   
      0                  1        1       1            1            1   
      1                  3        3       3            3            3   
 0   -1                  1        1       1            1            1   
      1                 14       14      14           14           14   
 1    1                 28       28      28           28           28   

               winner_draw  double_home_away  double_home_draw  \
real favorite                                                    
-1   -1                 29                29                29   
      0                  1                 1                 1   
      1                  3                 3                 3   
 0   -1                  1                 1                 1   
      1                 14                14                14   
 1    1                 28                28                28   

               double_away_draw  winner_margin  underdog  double_margin  \
real favorite                                                             
-1   -1                      29             29        29             29   
      0                       1              1         1              1   
      1                       3              3         3              3   
 0   -1                       1              1         1              1   
      1                      14             14        14             14   
 1    1                      28             28        28             28   

               predict_home_away  predict_home_draw  predict_away_draw  
real favorite                                                           
-1   -1                       29                 29                 29  
      0                        1                  1                  1  
      1                        3                  3                  3  
 0   -1                        1                  1                  1  
      1                       14                 14                 14  
 1    1                       28                 28                 28

In [80]:
new_prediction.groupby(['real', 'predict']).count()

home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real predict                                                                   
-1   -1                 14            14      14            14            14   
      0                 10            10      10            10            10   
      1                  9             9       9             9             9   
 0   -1                  1             1       1             1             1   
      0                  3             3       3             3             3   
      1                 11            11      11            11            11   
 1   -1                  1             1       1             1             1   
      0                  2             2       2             2             2   
      1                 25            25      25            25            25   

              predict_win  league  winner_home  winner_away  winner_draw  \
real predict                                                               
-1   -1                14      14           14           14           14   
      0                10      10           10           10           10   
      1                 9       9            9            9            9   
 0   -1                 1       1            1            1            1   
      0                 3       3            3            3            3   
      1                11      11           11           11           11   
 1   -1                 1       1            1            1            1   
      0                 2       2            2            2            2   
      1                25      25           25           25           25   

              double_home_away  double_home_draw  double_away_draw  \
real predict                                                         
-1   -1                     14                14                14   
      0                     10                10                10   
      1                      9                 9                 9   
 0   -1                      1                 1                 1   
      0                      3                 3                 3   
      1                     11                11                11   
 1   -1                      1                 1                 1   
      0                      2                 2                 2   
      1                     25                25                25   

              winner_margin  favorite  underdog  double_margin  \
real predict                                                     
-1   -1                  14        14        14             14   
      0                  10        10        10             10   
      1                   9         9         9              9   
 0   -1                   1         1         1              1   
      0                   3         3         3              3   
      1                  11        11        11             11   
 1   -1                   1         1         1              1   
      0                   2         2         2              2   
      1                  25        25        25             25   

              predict_home_away  predict_home_draw  predict_away_draw  
real predict                                                           
-1   -1                      14                 14                 14  
      0                      10                 10                 10  
      1                       9                  9                  9  
 0   -1                       1                  1                  1  
      0                       3                  3                  3  
      1                      11                 11                 11  
 1   -1                       1                  1                  1  
      0                       2                  2                  2  
      1                      25                 25                 25

In [82]:
gb = new_prediction[(new_prediction['league'] == 39)]
gb = gb[['home_team.id', 'away_team.id', 'predict', 'favorite', 'real']]
gb.head(10)

,home_team.id,away_team.id,predict,underdog,favorite,real
10,66,41,1,0,1,0
11,44,49,-1,0,-1,-1
12,36,60,0,1,1,-1
13,63,46,1,1,-1,-1
14,40,48,1,0,1,1
15,33,42,1,0,1,0
16,34,45,0,1,-1,-1
17,62,50,-1,0,-1,-1
18,47,51,1,0,1,1
19,39,52,1,0,1,0


In [100]:
gb = new_prediction[(new_prediction['league'] == 39)]
gb = gb[['home_team.id', 'away_team.id', 'predict', 'favorite', 'real']]
gb.head(10)

,home_team.id,away_team.id,predict,favorite,real
10,66,41,1,1,0
11,44,49,-1,-1,-1
12,36,60,0,1,0
13,63,46,1,-1,-1
14,40,48,1,1,1
15,33,42,1,1,0
16,34,45,0,-1,-1
17,62,50,-1,-1,-1
18,47,51,1,1,1
19,39,52,1,1,0


In [149]:
gb = new_prediction[(new_prediction['league'] == 39)]
gb = gb[['home_team.id', 'away_team.id', 'predict', 'favorite', 'real']]
gb.head(10)

,home_team.id,away_team.id,predict,favorite,real
10,66,41,1,1,-1
11,44,49,-1,-1,-1
12,36,60,0,1,-1
13,63,46,1,-1,-1
14,40,48,1,1,1
15,33,42,1,1,-1
16,34,45,0,-1,-1
17,62,50,-1,-1,-1
18,47,51,1,1,1
19,39,52,1,1,-1


In [214]:
gb = new_prediction[(new_prediction['league'] == 140)]
gb = gb[['home_team.id', 'away_team.id', 'predict', 'favorite', 'real']]
gb.head(10)

,home_team.id,away_team.id,predict,favorite,real
0,542,529,-1,-1,-1
1,531,536,1,-1,-1
2,543,797,1,1,1
3,538,548,0,-1,1
4,545,724,1,1,-1
5,715,539,1,1,-1
6,727,530,1,-1,-1
7,541,726,0,1,1
8,532,546,1,-1,0
9,533,720,1,1,1


In [ ]:
home_team= 500 Bologna - 2020, away_team= 490 Cagliari - 2020, result= 1
home_team= 501 Crotone - 2020, away_team= 499 Atalanta - 2020, result= -1
home_team= 505 Inter - 2020, away_team= 523 Parma - 2020, result= 1
home_team= 492 Napoli - 2020, away_team= 488 Sassuolo - 2020, result= 1
home_team= 497 AS Roma - 2020, away_team= 502 Fiorentina - 2020, result= 1
home_team= 498 Sampdoria - 2020, away_team= 495 Genoa - 2020, result= 1
home_team= 515 Spezia - 2020, away_team= 496 Juventus - 2020, result= 1
home_team= 503 Torino - 2020, away_team= 487 Lazio - 2020, result= 1
home_team= 494 Udinese - 2020, away_team= 489 AC Milan - 2020, result= -1
home_team= 504 Verona - 2020, away_team= 506 Benevento - 2020, result= 0
# Target is the favorite a winner?

In [44]:
print((47+64)/len(transformed_odds))
transformed_odds.groupby(['real', 'favorite']).count()

0.47844827586206895


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real favorite                                                                   
-1   -1.0                47            47      47            47            47   
      1.0                36            36      36            36            36   
 0   -1.0                24            24      24            24            24   
      1.0                40            40      40            40            40   
 1   -1.0                20            20      20            20            20   
      1.0                64            64      64            64            64   

               predict_win  predict  league  winner_home  winner_away  \
real favorite                                                           
-1   -1.0               47       47      47           47           47   
      1.0               36       36      36           36           36   
 0   -1.0               24       24      24           24           24   
      1.0               40       40      40           40           40   
 1   -1.0               20       20      20           20           20   
      1.0               64       64      64           64           64   

               winner_draw  double_home_away  double_home_draw  \
real favorite                                                    
-1   -1.0               47                47                47   
      1.0               36                36                36   
 0   -1.0               24                24                24   
      1.0               40                40                40   
 1   -1.0               20                20                20   
      1.0               64                64                64   

               double_away_draw  winner_margin  underdog  double_margin  \
real favorite                                                             
-1   -1.0                    47             47        47             47   
      1.0                    36             36        36             36   
 0   -1.0                    24             24        24             24   
      1.0                    40             40        40             40   
 1   -1.0                    20             20        20             20   
      1.0                    64             64        64             64   

               predict_home_away  predict_home_draw  predict_away_draw  
real favorite                                                           
-1   -1.0                     47                 47                 47  
      1.0                     36                 36                 36  
 0   -1.0                     24                 24                 24  
      1.0                     40                 40                 40  
 1   -1.0                     20                 20                 20  
      1.0                     64                 64                 64

In [43]:
print((33+10+53)/len(transformed_odds))
transformed_odds.groupby(['real', 'predict']).count()

0.41379310344827586


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real predict                                                                   
-1   -1                 33            33      33            33            33   
      0                 13            13      13            13            13   
      1                 38            38      38            38            38   
 0   -1                 20            20      20            20            20   
      0                 10            10      10            10            10   
      1                 34            34      34            34            34   
 1   -1                 20            20      20            20            20   
      0                 11            11      11            11            11   
      1                 53            53      53            53            53   

              predict_win  league  winner_home  winner_away  winner_draw  \
real predict                                                               
-1   -1                33      33           33           33           33   
      0                13      13           13           13           13   
      1                38      38           38           38           38   
 0   -1                20      20           20           20           20   
      0                10      10           10           10           10   
      1                34      34           34           34           34   
 1   -1                20      20           20           20           20   
      0                11      11           11           11           11   
      1                53      53           53           53           53   

              double_home_away  double_home_draw  double_away_draw  \
real predict                                                         
-1   -1                     33                33                33   
      0                     13                13                13   
      1                     38                38                38   
 0   -1                     20                20                20   
      0                     10                10                10   
      1                     34                34                34   
 1   -1                     20                20                20   
      0                     11                11                11   
      1                     53                53                53   

              winner_margin  favorite  underdog  double_margin  \
real predict                                                     
-1   -1                  33        32        33             33   
      0                  13        13        13             13   
      1                  38        38        38             38   
 0   -1                  20        20        20             20   
      0                  10        10        10             10   
      1                  34        34        34             34   
 1   -1                  20        20        20             20   
      0                  11        11        11             11   
      1                  53        53        53             53   

              predict_home_away  predict_home_draw  predict_away_draw  
real predict                                                           
-1   -1                      33                 33                 33  
      0                      13                 13                 13  
      1                      38                 38                 38  
 0   -1                      20                 20                 20  
      0                      10                 10                 10  
      1                      34                 34                 34  
 1   -1                      20                 20                 20  
      0                      11                 11                 11  
      1                      53                 53                 53

In [23]:
# How many underdogs i found?
underdogs = transformed_odds[(transformed_odds["underdog"] == 1)]
underdogs.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,winner_home,...,double_home_away,double_home_draw,double_away_draw,winner_margin,favorite,underdog,double_margin,predict_home_away,predict_home_draw,predict_away_draw
2,545,727,6,1.182168,1.240975,1.048746,0,0,140,0.370370,...,0.735294,0.714286,0.694444,-0.048531,1.0,1,-1.144024,2.230914,2.289721,2.423143
3,542,797,6,0.673380,2.285670,0.538878,-1,0,140,0.571429,...,0.751880,0.877193,0.500000,-0.056277,1.0,1,-1.129073,1.212259,2.824548,2.959050
7,724,533,7,0.554033,0.520371,2.293519,0,1,140,0.266667,...,0.751880,0.578035,0.800000,-0.057502,-1.0,1,-1.129914,2.847552,2.813889,1.074404
8,529,541,7,2.244724,0.509370,0.788361,-1,-1,140,0.512821,...,0.775194,0.800000,0.555556,-0.053756,1.0,1,-1.130749,3.033085,1.297731,2.754094
11,536,545,7,0.968180,1.230500,1.034634,-1,0,140,0.689655,...,0.833333,0.909091,0.380228,-0.063459,1.0,1,-1.122652,2.002814,2.265134,2.198680


In [24]:
underdogs.shape

(99, 22)

In [31]:
# How many times the underdog won?
print(44/99)
underdogs.groupby(['real', 'favorite']).count()

0.4444444444444444


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real favorite                                                                   
-1   -1.0                24            24      24            24            24   
      1.0                16            16      16            16            16   
 0   -1.0                10            10      10            10            10   
      1.0                15            15      15            15            15   
 1   -1.0                13            13      13            13            13   
      1.0                20            20      20            20            20   

               predict_win  predict  league  winner_home  winner_away  \
real favorite                                                           
-1   -1.0               24       24      24           24           24   
      1.0               16       16      16           16           16   
 0   -1.0               10       10      10           10           10   
      1.0               15       15      15           15           15   
 1   -1.0               13       13      13           13           13   
      1.0               20       20      20           20           20   

               winner_draw  double_home_away  double_home_draw  \
real favorite                                                    
-1   -1.0               24                24                24   
      1.0               16                16                16   
 0   -1.0               10                10                10   
      1.0               15                15                15   
 1   -1.0               13                13                13   
      1.0               20                20                20   

               double_away_draw  winner_margin  underdog  double_margin  \
real favorite                                                             
-1   -1.0                    24             24        24             24   
      1.0                    16             16        16             16   
 0   -1.0                    10             10        10             10   
      1.0                    15             15        15             15   
 1   -1.0                    13             13        13             13   
      1.0                    20             20        20             20   

               predict_home_away  predict_home_draw  predict_away_draw  
real favorite                                                           
-1   -1.0                     24                 24                 24  
      1.0                     16                 16                 16  
 0   -1.0                     10                 10                 10  
      1.0                     15                 15                 15  
 1   -1.0                     13                 13                 13  
      1.0                     20                 20                 20

In [32]:
# How many times the underdog won?
print(39/99)
underdogs.groupby(['real', 'predict']).count()

0.3939393939393939


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real predict                                                                   
-1   -1                 10            10      10            10            10   
      0                 13            13      13            13            13   
      1                 18            18      18            18            18   
 0   -1                  6             6       6             6             6   
      0                 10            10      10            10            10   
      1                  9             9       9             9             9   
 1   -1                 13            13      13            13            13   
      0                 11            11      11            11            11   
      1                  9             9       9             9             9   

              predict_win  league  winner_home  winner_away  winner_draw  \
real predict                                                               
-1   -1                10      10           10           10           10   
      0                13      13           13           13           13   
      1                18      18           18           18           18   
 0   -1                 6       6            6            6            6   
      0                10      10           10           10           10   
      1                 9       9            9            9            9   
 1   -1                13      13           13           13           13   
      0                11      11           11           11           11   
      1                 9       9            9            9            9   

              double_home_away  double_home_draw  double_away_draw  \
real predict                                                         
-1   -1                     10                10                10   
      0                     13                13                13   
      1                     18                18                18   
 0   -1                      6                 6                 6   
      0                     10                10                10   
      1                      9                 9                 9   
 1   -1                     13                13                13   
      0                     11                11                11   
      1                      9                 9                 9   

              winner_margin  favorite  underdog  double_margin  \
real predict                                                     
-1   -1                  10         9        10             10   
      0                  13        13        13             13   
      1                  18        18        18             18   
 0   -1                   6         6         6              6   
      0                  10        10        10             10   
      1                   9         9         9              9   
 1   -1                  13        13        13             13   
      0                  11        11        11             11   
      1                   9         9         9              9   

              predict_home_away  predict_home_draw  predict_away_draw  
real predict                                                           
-1   -1                      10                 10                 10  
      0                      13                 13                 13  
      1                      18                 18                 18  
 0   -1                       6                  6                  6  
      0                      10                 10                 10  
      1                       9                  9                  9  
 1   -1                      13                 13                 13  
      0                      11                 11                 11  
      1                       9                  9                  9

In [ ]:
# If this was efficient? wins underdog / Underdogs detected

In [ ]:
# Compare winner vs real (odds favorite) and predict vs real (footballmlp fav)